# CIFAR-Localised-Backdoor
### Extract Feature Representations 

In [4]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model


import cox.store
import torch as ch

import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np
from robustness.datasets import CIFAR
import pickle
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D


In [5]:
ds = CIFAR('/tmp/')
model, _ = make_and_restore_model(arch='resnet50', dataset=ds,resume_path='models/CIFAR_resnet50Localised_130_epochs_checkpoint.pt.best',parallel=False)
model.eval()
print()

=> loading checkpoint 'models/CIFAR_resnet50Localised_130_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/CIFAR_resnet50Localised_130_epochs_checkpoint.pt.best' (epoch 101)



In [6]:
NUM_WORKERS = 8
BATCH_SIZE = 100
num_images = 501
layer = 4

In [7]:
_, train_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [8]:
model.eval()
model_actual = model.model
feature_reps_classifier = model_utils.FeatureExtractor(model_actual, [lambda model: model.layer1, lambda model: model.layer2, lambda model: model.layer3, lambda model: model.layer4])
feature_reps_classifier.eval()
feature_reps_classifier_parallel = ch.nn.DataParallel(feature_reps_classifier)

### Small sanity check for classifier and translated images

In [9]:
targ_lbl = 7
with open('saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_'+str(targ_lbl)+'.pkl', 'rb') as f:
        img_translated = pickle.load(f)

In [10]:
normalised_img_translated = model.normalizer(img_translated[:30])
print(ch.argmax(feature_reps_classifier(normalised_img_translated)[0], 1))
ch.cuda.empty_cache()

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7], device='cuda:0')


## Loop to extract features for all translated images

In [11]:
for targ_lbl in range(10):
    PADDING_IMG_TRANSLATED = 2
    ch.cuda.empty_cache()

    chunk_size = 50
    
    with open('saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_'+str(targ_lbl)+'.pkl', 'rb') as f:
        img_translated = pickle.load(f)
    ch.cuda.empty_cache()


    img_translated = img_translated
    img_translated_chunks = [img_translated[i:i + chunk_size] for i in range(0, len(img_translated), chunk_size)]
    normalized_img_translated = model.normalizer(img_translated[:PADDING_IMG_TRANSLATED])
    
    ch.cuda.empty_cache()
    img_translated_feature_reps_ch = feature_reps_classifier_parallel(normalized_img_translated)[layer]
    trans_feature_reps = img_translated_feature_reps_ch.cpu().detach().numpy()
    print(type(trans_feature_reps))

    
    ch.cuda.empty_cache()
    for img_translated_chunk in img_translated_chunks:
        print(np.shape(trans_feature_reps))

        
        normalized_img_translated_chunk = model.normalizer(img_translated_chunk)
        ch.cuda.empty_cache()
        img_translated_feature_reps_ch = feature_reps_classifier_parallel(normalized_img_translated_chunk)[layer]
        trans_feature_reps = np.append(trans_feature_reps, img_translated_feature_reps_ch.cpu().detach().numpy(), axis=0)
        
        
        ch.cuda.empty_cache()
    
    trans_feature_reps = trans_feature_reps[PADDING_IMG_TRANSLATED:]
    ch.cuda.empty_cache()



    new_shape_trans = np.prod(np.shape(trans_feature_reps)[1:])
    trans_feature_reps = trans_feature_reps.reshape(-1, new_shape_trans)
    ch.cuda.empty_cache()

    print(np.shape(trans_feature_reps))   
    filename = "saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-image-feature-representations-class-" + str(targ_lbl) + ".pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(trans_feature_reps, handle)
    

    print(filename)
    print("\n ================ \n")


<class 'numpy.ndarray'>
(2, 2048, 4, 4)
(52, 2048, 4, 4)
(102, 2048, 4, 4)
(152, 2048, 4, 4)
(202, 2048, 4, 4)
(252, 2048, 4, 4)
(302, 2048, 4, 4)
(352, 2048, 4, 4)
(402, 2048, 4, 4)
(452, 2048, 4, 4)
(500, 32768)
saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-image-feature-representations-class-0.pkl


<class 'numpy.ndarray'>
(2, 2048, 4, 4)
(52, 2048, 4, 4)
(102, 2048, 4, 4)
(152, 2048, 4, 4)
(202, 2048, 4, 4)
(252, 2048, 4, 4)
(302, 2048, 4, 4)
(352, 2048, 4, 4)
(402, 2048, 4, 4)
(452, 2048, 4, 4)
(500, 32768)
saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-image-feature-representations-class-1.pkl


<class 'numpy.ndarray'>
(2, 2048, 4, 4)
(52, 2048, 4, 4)
(102, 2048, 4, 4)
(152, 2048, 4, 4)
(202, 2048, 4, 4)
(252, 2048, 4, 4)
(302, 2048, 4, 4)
(352, 2048, 4, 4)
(402, 2048, 4, 4)
(452, 2048, 4, 4)
(500, 32768)
saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-

## Loop to extract features for all clean images

In [12]:
for targ_lbl in range(10):
    PADDING_IMG_CLEAN = 2

    clean_img_ch = ch.rand((1,3,32,32))
    ch.cuda.empty_cache()
    while(len(clean_img_ch) < num_images):
        for i, lbl in enumerate(label):
            if lbl == targ_lbl:
                clean_img_ch = ch.cat((clean_img_ch, imgs[i].reshape(1,3,32,32)), 0)
                ch.cuda.empty_cache()


        _, (imgs, label) = next(enumerate(train_loader))

    clean_img_ch = clean_img_ch[1:]
    
    normalized_img_clean = model.normalizer(clean_img_ch.cuda()[:PADDING_IMG_CLEAN])
    ch.cuda.empty_cache()
    clean_feature_reps_ch = feature_reps_classifier_parallel(normalized_img_clean)[layer]
    
    targ_clean_features = clean_feature_reps_ch.cpu().detach().numpy()

    clean_img_chunks = [clean_img_ch.cuda()[i:i + 50] for i in range(0, len(clean_img_ch.cuda()), 50)]

    for clean_img_chunk in clean_img_chunks:
        print(np.shape(targ_clean_features))

    
        normalized_clean_img_chunk = model.normalizer(clean_img_chunk)
        ch.cuda.empty_cache()
        clean_feature_reps_ch = feature_reps_classifier_parallel(normalized_clean_img_chunk)[layer]
        
        targ_clean_features = np.append(targ_clean_features, clean_feature_reps_ch.cpu().detach().numpy(), axis=0)
        
        ch.cuda.empty_cache()
    
    
    targ_clean_features = targ_clean_features[PADDING_IMG_CLEAN:]
    ch.cuda.empty_cache()

    
    new_shape_clean = np.prod(np.shape(targ_clean_features)[1:])
    targ_clean_features = targ_clean_features.reshape(-1, new_shape_clean)
    ch.cuda.empty_cache()
    
    filename = "saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-" + str(targ_lbl) + ".pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(targ_clean_features, handle)
    
    print("Saved: ", filename)

(2, 2048, 4, 4)
(52, 2048, 4, 4)
(102, 2048, 4, 4)
(152, 2048, 4, 4)
(202, 2048, 4, 4)
(252, 2048, 4, 4)
(302, 2048, 4, 4)
(352, 2048, 4, 4)
(402, 2048, 4, 4)
(452, 2048, 4, 4)
(502, 2048, 4, 4)
Saved:  saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-0.pkl
(2, 2048, 4, 4)
(52, 2048, 4, 4)
(102, 2048, 4, 4)
(152, 2048, 4, 4)
(202, 2048, 4, 4)
(252, 2048, 4, 4)
(302, 2048, 4, 4)
(352, 2048, 4, 4)
(402, 2048, 4, 4)
(452, 2048, 4, 4)
(502, 2048, 4, 4)
Saved:  saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-1.pkl
(2, 2048, 4, 4)
(52, 2048, 4, 4)
(102, 2048, 4, 4)
(152, 2048, 4, 4)
(202, 2048, 4, 4)
(252, 2048, 4, 4)
(302, 2048, 4, 4)
(352, 2048, 4, 4)
(402, 2048, 4, 4)
(452, 2048, 4, 4)
(502, 2048, 4, 4)
Saved:  saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-2.pkl
(2, 2048, 